## Parsing JSON files from the Twitter API (Python)

This notebook was posted by Simon Lindgren // [@simonlindgren](http://www.twitter.com/simonlindgren) // [simonlindgren.com](http://simonlindgren.com).

The Twitter APIs, like many other services on the internet, output data in the interchange format [JSON](https://www.copterlabs.com/json-what-it-is-how-it-works-how-to-use-it/). 

This notebook is about how to [parse](https://en.wikipedia.org/wiki/Parsing) such data into the more accessible format of pandas dataframes for further analysis or file export.

In [ ]:
import json
import pandas as pd

In [ ]:
# define flattening function from https://gist.github.com/higarmi/6708779
def flattenDict(d, result=None):
    if result is None:
        result = {}
    for key in d:
        value = d[key]
        if isinstance(value, dict):
            value1 = {}
            for keyIn in value:
                value1[".".join([key,keyIn])]=value[keyIn]
            flattenDict(value1, result)
        elif isinstance(value, (list, tuple)):   
            for indexB, element in enumerate(value):
                if isinstance(element, dict):
                    value1 = {}
                    index = 0
                    for keyIn in element:
                        newkey = ".".join([key,keyIn])        
                        value1[".".join([key,keyIn])]=value[indexB][keyIn]
                        index += 1
                    for keyA in value1:
                        flattenDict(value1, result)   
        else:
            result[key]=value
    return result

In [ ]:
f = open("tweets.json", "r")
data_rows = [flattenDict(json.loads(line)) for line in f]

df = pd.DataFrame(data_rows)

In [ ]:
#list(df) # view available columns

In [ ]:
# get the desired columns by name
cols = list(['created_at']+           # time of tweet
        ['user.name']+         # about the author
        ['user.screen_name']+
        ['user.description']+
        ['user.location']+
        ['user.followers_count']+
        ['user.friends_count']+    
        ['in_reply_to_screen_name']+  # addressee
        ['entities.user_mentions.screen_name']+
        ['entities.hashtags.text']+  # tweet contents
        ['entities.media.type']+
        ['entities.media.url']+
        ['text']+
        ['favorite_count']+          # tweet metadata
        ['reply_count']+
        ['retweet_count']+
        ['quote_count']+
        ['lang'])

tweets_df = df[cols]
tweets_df.columns = ['time', 'user', 'user_sn', 'desc', 'loc', 'followers', 'friends', 
                     'to_user_sn','mentioning_sns', 'hashtags',
                     'media_type', 'media_url', 'text', 'faves', 'replies', 'retweets',
                     'quotes', 'lang']


tweets_df = tweets_df.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
tweets_df = tweets_df.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
tweets_df = tweets_df.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

tweets_df.head()

In [ ]:
# Filter by language
tweets_df = tweets_df.loc[tweets_df['lang'] == 'sv']
tweets_df.head()

In [ ]:
# Export to csv
tweets_df.to_csv("mega.csv")

##### Export custom csvs

In [ ]:
# time and text
cols = list(['time']+['text'])
tt_df = tweets_df[cols]
tt_df.head()
tt_df.to_csv("tt.csv")

In [ ]:
# source -> target
cols = list(['user_sn']+['to_user_sn'])
pairs_df = tweets_df[cols]
pairs_df = pairs_df[pd.notnull(pairs_df['to_user_sn'])] # take only rows where target is not empty
pairs_df.head()

##### Get a column as txt

In [ ]:
f = open("tweets.txt", 'w')
for tweet in tweets_df['text']:
    f.write(tweet + '\n')

In [ ]:
f = open("profiles.txt", 'w')
for desc in tweets_df['desc']:
    if desc is not None:
        f.write(desc + '\n')
    else:
        f.write('None\n')